In [1]:
# 8-1
!mkdir chap8
%cd ./chap8

/Users/maton/BERT_Practice/chap8


/Users/maton/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# 8-3
import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 8-4（説明コメント付き：Unicode 正規化 NFKC による表記ゆれの吸収）
# -----------------------------------------------------------------------------
# 目的：
#  - Unicode の **互換性分解 + 合成**（NFKC）で、全角/半角・互換文字（例：半角ｶﾅ・ローマ数字・① 等）
#    を正規化し、検索/重複排除/機械学習前処理での表記ゆれを減らす。
#
# 理論メモ（NFC/NFD/NFKC/NFKD の違い）：
#  - NFC: 正規分解 → 合成（互換性は無視）。見た目を保ちながら正規の合成形式へ。
#  - NFD: 正規分解のみ（結合文字にバラす）。検索・照合の下処理に使うことがある。
#  - NFKC: **互換分解**（見た目は同じでも “意味的に同一視される” 文字を分解）→ 合成。
#          例：全角英数/記号、半角ｶﾅ、ローマ数字 Ⅳ、丸数字 ① などを通常の文字に畳み込む。
#  - NFKD: 互換分解のみ。
# 互換分解は “情報の落ち” が起こり得る（例：①→1、㍍→メートル→m など）。監査用途では注意。
# -----------------------------------------------------------------------------

import unicodedata

# NFKC で正規化する関数
# - 全角英数→半角、半角ｶﾅ→全角カタカナ、結合記号の統合 等を一括で行う
normalize = lambda s: unicodedata.normalize("NFKC", s)

# 動作例：全角/半角の統一（学習前の前処理・検索キー作成などで有効）
print(f'ＡＢＣ -> {normalize("ＡＢＣ")}')  # 全角アルファベット → 半角 "ABC"
print(f'ABC -> {normalize("ABC")}')  # 既に半角 → 変化なし
print(f'１２３ -> {normalize("１２３")}')  # 全角数字 → 半角 "123"
print(f'123 -> {normalize("123")}')  # 既に半角 → 変化なし
print(f'アイウ -> {normalize("アイウ")}')  # 全角カタカナ → 変化なし
print(f'ｱｲｳ -> {normalize("ｱｲｳ")}')  # 半角ｶﾅ → 全角カタカナ "アイウ"

# -----------------------------------------------------------------------------
# 追加の知見（必要ならテストして確認）：
#  - 濁点付き半角ｶﾅ（例：ｶﾞ）も結合が解決され全角「ガ」に統一される：
#      normalize("ｶﾞ") == "ガ"
#  - 互換文字の折り畳み：
#      normalize("ⅠⅡⅢ") -> "III"  （ローマ数字 → ラテン大文字）
#      normalize("①②")   -> "12"   （丸数字 → 通常数字）
#  - 正規化は “可逆でない” ことがあるため、**原文は別項目で保存**しておくと安全。
# -----------------------------------------------------------------------------

ＡＢＣ -> ABC
ABC -> ABC
１２３ -> 123
123 -> 123
アイウ -> アイウ
ｱｲｳ -> アイウ
